In [1]:
import copy
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords

from gensim import corpora
import gensim

In [2]:
# 문장 단위로 나누고 감정분석된 데이터를 불러옴
data = pd.read_csv('data/sentiment_sentences_에어서울.csv')

In [3]:
# 부정인 데이터만 사용
data = data[data['sentiment']=='부정']

In [4]:
kiwi = Kiwi(typos='basic')
stopwords = Stopwords()

In [5]:
def preprocess_korean(text, analyzer=kiwi, stopwords=stopwords):
    my_text = copy.copy(text)
    my_text = my_text.replace('\n', ' ')  # (1) 줄바꿈 제거
    my_text = kiwi.space(my_text)         # (2) 띄어쓰기 교정
    sents = kiwi.split_into_sents(my_text)  # (3) 문장 단위 분리
    p = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣]')     # (4) 특수 문자 제거
    all_result = []

    for sent in sents:
        token_result = kiwi.tokenize(sent.text, stopwords=stopwords)  # (5) 형태소 분석
        for i in token_result:
            if i.tag in ['VA', 'NNP', 'NNG','VV']:  # (6) 형용사, 명사, 동사 필터링
                word = p.sub('', i.form)       # (7) 특수 문자 제거
                if word:  # 빈 문자열 제거
                    all_result.append(word)

    return all_result 

In [6]:
data['preprocessed_review'] = data['sentence'].apply(lambda x: preprocess_korean(x))

In [7]:
data.head()

,date,sentence,sentiment,preprocessed_review
0,2025-05-02,정말 저렴한 가격으로 이용한 에어서울 티웨이는 좌석이 좁아 정말 불편했는데 에어 서...,부정,"[저렴하, 가격, 이용, 에어, 서울, 티웨이, 좌석, 불편, 에어, 서울, 좌석,..."
8,2024-10-27,가는 길에 시간 돌아오는 길에 시간씩 일방적으로 비행기가 지연되었습니다 이해할 수 ...,부정,"[가, 길, 시간, 돌아오, 길, 시간, 일방, 비행기, 지연, 이해]"
9,2024-10-24,공항으로 가는 길에는 좌석이 멀리 떨어져 있어서 아쉬웠지만 돌아오는 길에는 나란히 ...,부정,"[공항, 가, 길, 좌석, 떨어지, 돌아오, 길, 줄, 서, 있]"
18,2024-08-29,출발과 귀국 모두 분씩 출발이 지연되었습니다,부정,"[출발, 귀국, 출발, 지연]"
20,2024-08-27,지연이 있긴 했지만 비행은 좋았어요,부정,"[지연, 있, 비행, 좋]"


In [8]:
word_dict = corpora.Dictionary(data['preprocessed_review'])
corpus = [word_dict.doc2bow(text) for text in data['preprocessed_review']]

In [9]:
N_TOPICS = 3
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics = N_TOPICS, 
                                           id2word=word_dict,
                                           passes=10,
                                           random_state=42 )

In [10]:
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.025*"있" + 0.020*"서비스" + 0.016*"문제" + 0.016*"여행" + 0.015*"비행기"')
(1, '0.057*"지연" + 0.052*"시간" + 0.029*"비행기" + 0.024*"출발" + 0.022*"항공편"')
(2, '0.037*"수하물" + 0.028*"길" + 0.020*"위탁" + 0.018*"좌석" + 0.017*"서울"')


# 시각화

In [17]:
from IPython.core.display import HTML

In [18]:
import pyLDAvis
from pyLDAvis import gensim_models

In [19]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, word_dict)
pyLDAvis.display(vis)